# Overview

In the figure below, we depict a high-level overview of the future used process to generate the Extended and Enriched Scenarios. 


In [ ]:
from IPython.display import Image
Image("image/process_overview.png")

The proccess of collecting input is done in two parts. First part is collecting input from a cloud database, which is currently imnplemted to read from MES database. Second part is collecting the csv files from inside of the calculator, as files that placed there once a year, and we call it **static data**. This demo will present the **processing of the static data into dynamic data and contructing extended time series**

## Reading the static data
First, let's see some example of the static data that we read

In [ ]:
# Make this step idempotent
import os
if 'OLD_WD' not in globals():
    OLD_WD = os.getcwd()
else:
    os.chdir(OLD_WD)
os.chdir('..')
# os.chdir("C:\\Users\\ShabtaiHS\\PycharmProjects\\MSR.ScenarioCalculator.Py")
os.getcwd()
# Example of possible output
# 'C:\\Users\\ShabtaiHS\\PycharmProjects\\MSR.ScenarioCalculator.Py'

In [ ]:
from scenario_calculator.io.base_reader import read_static_data
from scenario_calculator.settings import Settings
read_static_data(Settings.STATIC_DATA_DIR, Settings.FILE_MAPPING)["agriland_org"]

This is the **Agriland static data**.

As one can see, the data does not depends on date. Hence, we call it static and we can use it for a whole year.

In [ ]:
read_static_data(Settings.STATIC_DATA_DIR, Settings.FILE_MAPPING)["nbrof_years_extension"]

In [ ]:
df_model_info = read_static_data(Settings.STATIC_DATA_DIR, Settings.FILE_MAPPING)["model_info"]
df_model_info

In [ ]:
df_model_info.model_name[0]

In [ ]:
type(df_model_info.model_name[0])

In [ ]:
type(read_static_data(Settings.STATIC_DATA_DIR, Settings.FILE_MAPPING)["nbrof_years_extension"].iloc[0].item())

## Processing the static data into dynamic data and extending
Now, let's see some mocked data from the cloud. Note, the reader from the cloud doesn't exist yet thus, it is mocked.

In [ ]:
from scenario_calculator.io.reader import _join_frames_without_agriland
from scenario_calculator.io.base_reader import read_dynamic_data
from scenario_calculator.settings import Settings
from scenario_calculator.constants import DYNAMIC_DATA_CSV_BASENAME
dynamic_data_csv_url = (Settings.MOCK_CLOUD_DIR  / DYNAMIC_DATA_CSV_BASENAME).as_uri()
dynamic_data_first20 = read_dynamic_data(dynamic_data_csv_url).head(20)
dynamic_data_first20

Here is some information we can get from the dynamic file

In [ ]:
type(dynamic_data_first20["date"].iloc[0])

In [ ]:
type(dynamic_data_first20["scenario"].iloc[0])

In [ ]:
type(dynamic_data_first20["requesting_institution"].iloc[0])

The mocked cloud data contains information which is date dependant. Our first goal is to combain this information from the cloud to our static data, and get a dictionary of dataframes which are **date dependent, thus dynamic** which we can then feed to the calculator.


## Step 1: Processing the static data with the extension static data without Agriland
Agriland static data contains only information for few countries. Therefor there could be a situation where the dynamic Agriland data will give us empty dataframe. As a result of this information, the approach to that was to first process and extended the data without the Agriland dataframe, and only after that stage to add Agriland, if it exists.

### Example: extension dataframe without Agriland

In [ ]:
from scenario_calculator.io.reader import _join_frames_without_agriland
from scenario_calculator.io.base_reader import read_dynamic_data, read_static_data
from scenario_calculator.settings import Settings
real_local_path = Settings.STATIC_DATA_DIR
dynamic_data = read_dynamic_data(dynamic_data_csv_url)
frames = _join_frames_without_agriland(dynamic_data, read_static_data(real_local_path, Settings.FILE_MAPPING))
frames["extension"].head(10)

In [ ]:
# First 5 element of each column
for column in frames["extension"].columns:
    print(frames["extension"][column].iloc[:5])

Agriland dataframe is not yet processed in this stage as one can see:

In [ ]:
import sys
print(f'frames.keys(): {frames.keys()}', file=sys.stderr)
assert 'agriland' not in frames.keys()

## Step 2: Processing the static data and extension static data with Agriland
Let's proceed to processing the Agriland dataframe, taking in consideration that it might be empty

In [ ]:
from scenario_calculator.io.pre_process import \
    get_reporting_date, \
    calc_scenario_start_date, get_scenario_name, get_scenario_end_date, \
    get_end_year_extension

from scenario_calculator.io.agriland_parameters import get_original_agriland_frame
from scenario_calculator.utility import generate_quarter_format

reporting_date = get_reporting_date(dynamic_data)
scenario_name = get_scenario_name(dynamic_data)
start_date = calc_scenario_start_date(reporting_date)

# Creates dynamic Agriland dataframe from the static Agriland
frames["agriland"] = get_original_agriland_frame(frames["agriland_org"], scenario_name, generate_quarter_format(start_date))
frames["agriland"]


We have recieved the dynamic agriland. And now we can join it to the extension dataframe

In [ ]:
from scenario_calculator.io.reader import _agriland_join_frames

# Join the Agriland dataframe only if it's available:
frames["extension"] = _agriland_join_frames(frames["extension"],frames["agriland"])
frames["extension"].head(10)

In [ ]:
# Let's see the additional three columns
frames["extension"].columns


In [ ]:
for column in frames["extension"].columns:
    print(frames["extension"][column].iloc[:10])

That is the end of input proccessing with the static data

# Playgroud
This is were one can try out the calculator. For the moment it reads and write localy.

In [ ]:
from datetime import datetime

from scenario_calculator.constants import OUTPUT_DATA_CSV_BASENAME_FMT
from scenario_calculator.log import config_logger
from scenario_calculator.config import GeneralConfig, RunParams, RunDefinition
from scenario_calculator.calculator import calculate
from scenario_calculator.utility import make_datetime_filestamp

current_datetime = datetime.utcnow()
config_logger()
# We have already `dynamic_data_csv_url`
gbv_csv_file = OUTPUT_DATA_CSV_BASENAME_FMT.format(datetime_str=make_datetime_filestamp(current_datetime))
scenario_output_url_str = (Settings.DATA_OUT_DIR / gbv_csv_file).as_uri()
event_logger_url_str = None
run_params = RunParams(dynamic_data_csv_url, scenario_output_url_str, event_logger_url_str)
# set random RunDefinition parameters
run_definition = RunDefinition(-1, 0, "201709 Plus", "RABONETEU\\DoeJ", "7.8.11-beta.23", datetime(2018, 9, 30, 13, 53, 5, 608), "Scenario for Demo", current_datetime)
general_config = GeneralConfig()

# Let's see output of the calculator:
calculate(general_config, run_params, run_definition)

In [ ]:
# Here we can see the directory of the output and read the first 10 lines of the Scenario TimeSeries SBV csv file 
# that was calculated.
import sys
import pandas as pd
from scenario_calculator.settings import Settings
latest_written_scenario_tseries = sorted(Settings.DATA_OUT_DIR.glob('Mes_STIB_[0-9]*.csv'), reverse=True)[0]
print(f'latest_written_scenario_tseries: {latest_written_scenario_tseries}', file=sys.stderr)
result_tseries = pd.read_csv(str(latest_written_scenario_tseries), delimiter="|", encoding="utf-8")
result_tseries.head(10)

In [ ]:
# Here we can see the directory of the output and read the first 10 lines of the Scenario Info csv file 
# that was calculated. Of course there is only one line
latest_written_scenario_info = sorted(Settings.DATA_OUT_DIR.glob('Mes_STIB_SCENARIO_INFO_[0-9]*.csv'), reverse=True)[0]
print(f'latest_written_scenario_info: {latest_written_scenario_info}', file=sys.stderr)
result_info = pd.read_csv(str(latest_written_scenario_info), delimiter="|", encoding="utf-8")
result_info.head(10)

In [ ]:
# Here we can see the directory of the output and read the last 10 lines of the Scenario TimeSeries GBV csv file 
# that was calculated. 

latest_written_scenario_tseries_gbv = sorted(Settings.DATA_OUT_DIR.glob('Mes_STIB_GBV_[0-9]*.csv'), reverse=True)[0]
print(f'latest_written: {latest_written_scenario_tseries_gbv}', file=sys.stderr)
result_GBV = pd.read_csv(str(latest_written_scenario_tseries_gbv), delimiter="|", encoding="utf-8")
result_GBV.tail(5)
#result_GBV["Mapped_To_Variable_Code"].unique()
#result_GBV.columns[18]

In [ ]:
result_GBV.head(5)

In [ ]:
# Here is a demonstration of filtering agriland dataframe on the Plus filter,
# since in the previous demonstration we only see filteration on Baseline

from scenario_calculator.io.agriland_parameters import get_original_agriland_frame
from scenario_calculator.settings import Settings
# get_original_agriland_frame(pd.Dataframe(), '201806 Plus', '2018Q4')
get_original_agriland_frame(frames["agriland_org"], '201806 Plus', '2018Q4')

In [ ]:
# Here is what happens when agriland does not exsist
import numpy as np
import pandas as pd
get_original_agriland_frame(pd.DataFrame(), '201806 Plus', '2018Q4')

## Dates in the pre_process
The following globals are a result of the pre process file that provides global variables that are fetched or
calculated from values in the cloud sql data base and then later used in the
transformation of static data to dynamic or even in the extension and
enrichment.

In [ ]:
from scenario_calculator.io.pre_process import get_reporting_date
from scenario_calculator.settings import Settings
type(get_reporting_date(dynamic_data))
# numpy.int64

In [ ]:
rd = get_reporting_date(dynamic_data)
rd
# 20180930

In [ ]:
from scenario_calculator.io.pre_process import calc_scenario_start_date
ssd = calc_scenario_start_date(rd)
ssd
# 20181231

In [ ]:
type(ssd)
# int

In [ ]:
from scenario_calculator.io.pre_process import get_end_year_extension
eds = get_scenario_end_date(rd, frames["nbrof_years_extension"], Settings.DEBUG_MODE)
eye = get_end_year_extension(eds)
eye
# 2048

In [ ]:
type(eye)
# numpy.int64

In [ ]:
from scenario_calculator.io.pre_process import get_scenario_name
sn = get_scenario_name(dynamic_data)
sn
# '201809 Baseline'